In [40]:
import pickle
import dill
from nltk.tag.hmm import HiddenMarkovModelTagger as hmm_tag
from collections import Counter
from nltk.metrics import ConfusionMatrix
from nltk.tokenize import TreebankWordTokenizer as twt
import numpy as np
import nltk

In [3]:
with open("pickled/dev.pickle", "rb") as file:
    dev = pickle.load(file)

In [4]:
with open("pickled/test.pickle", "rb") as file:
    test = pickle.load(file)

In [5]:
with open("pickled/train.pickle", "rb") as file:
    train = pickle.load(file)

In [6]:
train_bio = [train[i][:2] for i in range(len(train))]
dev_bio = [dev[i][:2] for i in range(len(dev))]
test_bio = [test[i][:2] for i in range(len(test))]

In [7]:
index = []
for i in range(len(train_bio)-1):
    if train_bio[i][0][-1] == '.' and train_bio[i+1][0][0].isupper():
        index.append(i+1)

train_sents = [train_bio[:index[0]]]
for i in range(len(index)-1):
    sent = train_bio[index[i]:index[i+1]]
    train_sents.append(sent)

In [8]:
index = []
for i in range(len(test_bio)-1):
    if test_bio[i][0][-1] == '.' and test_bio[i+1][0][0].isupper():
        index.append(i+1)

test_sents = [test_bio[:index[0]]]
for i in range(len(index)-1):
    sent = test_bio[index[i]:index[i+1]]
    test_sents.append(sent)

In [9]:
tagger = hmm_tag.train(train_sents, test_sequence=test_sents)

accuracy over 21110 tokens: 76.15


In [42]:
with open('hmm_tagger.dill', 'wb') as f:
    dill.dump(tagger, f)

In [35]:
words = []
tags = []
for sent in range(len(test_sents)):
    for word in range(len(test_sents[sent])):
        words.append(test_sents[sent][word][0])
        tags.append(test_sents[sent][word][1])

In [10]:
tagsFDist = nltk.FreqDist(tags)
tagsFDist

FreqDist({'O': 16693, 'I': 2522, 'B': 1895})

In [24]:
tagged_words = tagger.tag(words)
ref = [tagged_words[i][1] for i in range(len(tagged_words))]

In [12]:
#tags = len(tagged_words)*['O']
cm = ConfusionMatrix(ref, tags)
print(cm)

  |     B     I     O |
--+-------------------+
B |  <750>  171   740 |
I |   496 <1552> 2206 |
O |   649   799<13747>|
--+-------------------+
(row = reference; col = test)



In [13]:
labels = set('B I O'.split())

true_positives = Counter()
false_negatives = Counter()
false_positives = Counter()

for i in labels:
    for j in labels:
        if i == j:
            true_positives[i] += cm[i,j]
        else:
            false_negatives[i] += cm[i,j]
            false_positives[j] += cm[i,j]

f1_ind = []
for i in sorted(labels):
    if true_positives[i] == 0:
        fscore = 0
    else:
        precision = true_positives[i] / float(true_positives[i]+false_positives[i])
        recall = true_positives[i] / float(true_positives[i]+false_negatives[i])
        fscore = 2 * (precision * recall) / float(precision + recall)
    f1_ind.append(fscore)
    print(i, fscore)

B 0.42182227221597296
I 0.4580873671782763
O 0.8622052182639237


In [17]:
f1_mean = (1895*f1_ind[0]+2522*f1_ind[1])/(2522+1895)
f1_mean

0.4425287629325066

In [18]:
#'I': 2522, 'B': 1895